# Random Forest for An Alignment Cost-Based Classification of Log Traces

> Paper : <u>An Alignment Cost-Based Classification of Log Traces Using Machine-Learning </u><br>
> Date : June 2020 <br>
> Authors : <i>Mathilde Boltenhagen, Benjamin Chetioui, and Laurine Huber  </i> <br>

This notebook is organized as follow : <br> <br>
<b>0. Fitness function </b> 
- The lower bound fitness is a good contribution of the paper, please see the paper for more details. <br>

<b>1. Preprocessing the data:</b>
 - A function ```cleanDataForRF``` contains all the preprocessing steps. It reads the file, create the B.O.W. and the targets. 
 
<b>2. Model: </b> 
   - This is just the setting of the random forest learning model. 
   
<b>3. Cross-Validation of the method : </b>
 - A function ```runKFoldForRF``` runs a Kfold method to fit and test the model on the B.O.W.
 
<b> 4. Train and Test :</b>
- Train and test for many  m_AC values

## 0. Fitness

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd 
import numpy as np
np.random.seed(0)
import time

from statistics import mean

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow import enable_eager_execution
enable_eager_execution() 

The commented lines were used when the frequency of the variants was incorporated in the computation of fitness.
In fact, we decided to compute the fitness and lower bound for fitness on the variants only due to the understanding of the approach.

In [2]:
def fitness(packageForFitness, minRunLength):
    '''
    This function computes the fitness for all the sequences. 
    @sequences: the sequences of words
    @costs: the real alignment cost
    @minRunLength: the minimal run in the alignment dataset
    '''
    sumTraceFitness = 0
    totTraces = 0 
    for i in packageForFitness.index:
        sumTraceFitness += (1 - (packageForFitness.realCosts[i] / ( packageForFitness.lengths[i]  + minRunLength )))#*packageForFitness.freqs[i]
        #totTraces += packageForFitness.freqs[i]
        totTraces +=1
    return sumTraceFitness / totTraces

def LB_fitness(packageForFitness, minRunLength, m_AC, indices):
    '''
    This function computes lower bound of the fitness given in the paper. 
    @sequences: the sequences of words
    @minRunLength: the minimal run in the alignment dataset
    @m_AC: needed for the lowerbound formula
    @indices: if we compute the lower bound, then we don't iterate on all the traces but only the positives. 
    '''
    sumTraceFitness = 0
    totTraces = 0 
    for i in indices:
        sumTraceFitness += (1 - ((m_AC) / ( packageForFitness.lengths[i] + minRunLength )))#*packageForFitness.freqs[i]
    for i in packageForFitness.index:
        #totTraces += packageForFitness.freqs[i]
        totTraces +=1
    return sumTraceFitness / totTraces
        

## 1. Preprocessing the data

This function takes as input an alignment dataset and its Maximal Alignment Cost and clean the data in order to get a B.O.W. and the target classes. Please, see the definition of Maximal Alignment Cost Classification for more details.

In [3]:
# Initialize the "CountVectorizer" object as a GLOBAL VARIABLE, which is scikit-learn's bag of words tool.
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 5000)

In [4]:
def cleanDataForRF(dataFile,m_AC,vectorizer_already_trained=None): 
    '''
    Reads the file (1), specifies the target classes (2) and prepare the Bag of Words(3).
    @dataFile: (String) filename of the alignment dataset
    @m_AC: (int) maximal alignment cost classifier
    '''
    # ---- (1) yes, a bit of copy/paste... Read the file 
    data = pd.read_csv(dataFile,sep = ";", 
                   names = ["traces","tracesWithMoves","runs","runsWithMoves","costs","frequencies"])
    
    # ---- (2) create the positive and negative target depending on the m_AC parameter
    # alignment cost which interests us is greater than 10000 (other costs are just silent moves)
    # set the fitting to tmp_pos to set them latter to 1
    y = ((data["costs"] / 10000) / (m_AC+1)).astype(int)
    max_y = y.max()
    y = y.replace(0,"tmp_pos")
    y = y.replace(range(1,max_y + 1), 0)
    y = y.replace("tmp_pos",1)
    y = np.eye(2)[y.to_numpy().reshape(-1)]
    
    # ---- (3) prepare the Bag of Words
    traces_to_matrix = data.traces.str.split(":::",expand=True,)
    # this line takes the matrix of words, and transformed it to a list of sentences
    data_to_fit = [' '.join( [e.replace(" ","") for e in filter(None,a)]) for a in traces_to_matrix.values.tolist()]
    
    if vectorizer_already_trained :
        x = vectorizer.transform(data_to_fit).toarray()
    else :
        # then we can transform our data with the counter (it's like a one-hot-encode, right?)
        x = vectorizer.fit_transform(data_to_fit).toarray()
    
    # for fitness computation
    minLengthRun= len(data.runs.str.split(":::").min())
    lengths = data.traces.str.split(":::",expand=False,).str.len()-1
    realCosts = (data["costs"] / 10000).astype(int)
    packageForFitness = pd.DataFrame({"lengths":lengths, "realCosts": realCosts, "freqs": data.frequencies})
    
    return x, y, packageForFitness, minLengthRun, m_AC

In [5]:
# example of use
x, y, packageForFitness, minLengthRun, m_AC = cleanDataForRF("alignments/A_2012_im.csv",4)
x, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 5, 5, 0],
        [1, 0, 0, ..., 5, 3, 0],
        [1, 0, 0, ..., 3, 0, 0]], dtype=int64), array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]]))

## 2. Model

Just a sklearn call. In fact, the call will be in the KFold so we can keep the best model of prediction.

## 3.  Cross-Validation of the method 

From ```KFold``` of sklearn, we do a cross-validation on the training set. The outputs are average of the accuracy and the loss (binary-cross entropy). We use the ```BinaryCrossentropy``` function of <b> Tensorflow </b> to be consistent with the other experiment. 

- In order to see if positives or negatives items have better results, we give the results for `all` the test items, only the `positive` items and only the `negative` items. 
- `loss`, `acc` and `accLossPercentage` functions have been implemented in order to reduce number of lines

In [6]:
def loss(forest, x_test, y_test):
    y_test_predict_with_proba = forest.predict_proba(x_test)[1]
    return  BinaryCrossentropy()(y_test,y_test_predict_with_proba).numpy()

def acc(forest, x_test, y_test):
    y_test_predict = forest.predict(x_test)
    return accuracy_score(y_test,y_test_predict) 

In [7]:
def accLossPercentage(forest, x, y, indices_of_test, accArr, lossArr, percentageArr=None, freqs=None,classToTest=None):
    '''
    This function fills the arrays of results accArr, lossArr and percentageArr for all the test items, or only the 
    negative items (classToTest=0) or only the positive items (classToTest=1). percentageArr is optional because it 
    is not required for the entire dataset, i.e., when we do not specify the class. 
    This works by using a dynamic programmation for the arrays. The return element is either the current accuracy in 
    case of classToTest=None, either the indices in case of classToTest!=None.
    Params:
    @forest: a trained model
    @x_test: the dataset to test
    @y_test: the target value to predict for the test dataset
    @accArr: a list of the previous accurary, or an empty list
    @lossArr: a list of the previous loss, or an empty list
    @percentageArr:  a list of the previous percentage, or an empty list
    @classToTest: 1 or 0 for positive and negative. This will find the indices of the items that belongs to the class
    '''
    if classToTest!=None:
        indices = [i for i in indices_of_test if y[i][1]==classToTest]
        if len(indices)>0:
            accArr.append(acc(forest, x[indices], y[indices]))
            lossArr.append(loss(forest, x[indices], y[indices]))
        #percentageArr.append(freqs[indices].sum()/freqs[indices_of_test].sum())
        percentageArr.append(len(indices)/len(indices_of_test))
        return indices
    else :
        accuracy = acc(forest, x[indices_of_test], y[indices_of_test])
        accArr.append(accuracy)
        lossArr.append(loss(forest, x[indices_of_test], y[indices_of_test]))
        return accuracy

In [8]:
def runKFoldForRF(numberOfFold,x,y,packageForFitness,minLengthRun, m_AC):
    accAll, lossAll = [], []
    accNeg, lossNeg, percentageNeg = [], [], []
    accPos, lossPos, percentagePos = [], [], []
    
    realFitness, realLBFitness, predictedLBFitness = [], [], []
    packageForFitness.reset_index(drop=True,inplace=True)
    
    runtime = []

    # use a K-fold Cross-Validation and show average Loss and average Accuracy
    kfold = KFold(numberOfFold,)
    bestmodel, bestAccuracy = None, 0
    
    for indices_of_train, indices_of_test in kfold.split(x):
        forest = RandomForestClassifier(n_estimators = 100) 
        forest.fit(x[indices_of_train], y[indices_of_train])
        
        # compute loss and accuracy on the test items
        current_accuracy = accLossPercentage(forest, x, y, indices_of_test, accAll, lossAll, )
        
        # compute loss and accuracy on the test items that are negatives
        accLossPercentage(forest, x, y, indices_of_test, accNeg, lossNeg, percentageNeg,packageForFitness.freqs, 0)

        # compute loss and accuracy on the test items that are positives
        indices_of_pos = accLossPercentage(forest, x, y, indices_of_test, accPos, lossPos, percentagePos,packageForFitness.freqs, 1) 

        # compute fitness and lower-bound
        realFitness.append(fitness(packageForFitness.iloc[indices_of_test], minLengthRun))
        
        # compute real LB, which depends on number of pos items
        if len(indices_of_pos)>0:
            realLBFitness.append(LB_fitness(packageForFitness.iloc[indices_of_test], minLengthRun, m_AC, indices_of_pos))
        
        # compute predicted LB fitness and runtime
        start = time.time()
        predictions = forest.predict(x[indices_of_test])
        runtime.append((time.time()-start)/len(indices_of_test))
        
        indices_of_predicted_as_positives = [indices_of_test[i] for i in range(0,len(predictions)) if predictions[i][1]==1]
        if len(indices_of_predicted_as_positives)>0:
            predictedLBFitness.append(LB_fitness(packageForFitness.iloc[indices_of_test], minLengthRun, m_AC, indices_of_predicted_as_positives))
            
        if bestAccuracy < current_accuracy:
            bestmodel = forest
            bestAccuracy = current_accuracy
            
    print("[CROSS-VALIDATION]\n[ALL] Loss:", "{:.3f}".format(mean(lossAll)), "\t Acc:", "{:.3f}".format(mean(accAll)))
    print("[POSITIVE ({:.2f}%)] Loss:".format(mean(percentagePos)), "{:.3f}".format(mean(lossPos)), "\t Acc:", "{:.3f}".format(mean(accPos)))
    print("[NEGATIVE ({:.2f}%)] Loss:".format(mean(percentageNeg)), "{:.3f}".format(mean(lossNeg)),"\t Acc:", "{:.3f}".format(mean(accNeg)))
    print("Fitness {:.3f}".format(mean(realFitness)), "\t LB Fitness:", "{:.3f}\n".format(mean(realLBFitness)),"\t LB Fitness:", "{:.3f}\n".format(mean(predictedLBFitness)))
    print("Runtime (prediction per trace):{:.10f}".format(mean(runtime)))
    return bestmodel

In [9]:
# example of use
runKFoldForRF(3,x,y,packageForFitness, minLengthRun, m_AC)

Instructions for updating:
Use tf.cast instead.
[CROSS-VALIDATION]
[ALL] Loss: 0.011 	 Acc: 0.997
[POSITIVE (1.00%)] Loss: 0.003 	 Acc: 1.000
[NEGATIVE (0.00%)] Loss: 2.006 	 Acc: 0.246
Fitness 0.950 	 LB Fitness: 0.896
 	 LB Fitness: 0.898

Runtime (prediction per trace):0.0000212629


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## 4. Train and Test

In this section, we launch the cross-validation on a training set and test with a different set, thanks to `train_test_split`. 

The loop gives the results for <b>different mAC values</b> from 1 to 10. 

In [10]:
filename = "A_2012_shm.csv"

for i in [2,4,6,8,10]:
    print("\n","m_AC=",i)
    
    # read the datafile, packageForFitness, minLengthRun, m_AC are needed to compute fitness and LBfitness
    x, y, packageForFitness, minLengthRun, m_AC  = cleanDataForRF("alignments/"+filename,i)
    
    # split the dataset in TRAIN and TEST sets
    X_train, X_test, y_train, y_test, packageForFitness_train, packageForFitness_test = train_test_split(x, y, packageForFitness, test_size=0.33, random_state=42)

    # ----------------------------------------------
    #                     TRAIN 
    # ----------------------------------------------
    # run the cross-validation
    bestmodel = runKFoldForRF(10, X_train, y_train, packageForFitness_train, minLengthRun, m_AC)
    
    # ----------------------------------------------
    #                     TEST 
    # ----------------------------------------------
    # use the same function as in the cross-validation but for the test set. 
    accAll, lossAll = [], []
    accNeg, lossNeg, percentageNeg   = [], [], []
    accPos, lossPos, percentagePos = [], [], []
    
    packageForFitness_test.reset_index(drop=True,inplace=True)
    print(packageForFitness_test.freqs.shape)
    #compute loss and accuracy on the test items 
    accLossPercentage(bestmodel, X_test, y_test, list(range(0,len(y_test))), accAll, lossAll)
        
    # compute loss and accuracy on the test items that are negatives
    accLossPercentage(bestmodel, X_test, y_test, list(range(0,len(y_test))), accNeg, lossNeg, percentageNeg,packageForFitness_test.freqs, 0)

    # compute loss and accuracy on the test items that are positives
    indices_of_pos = accLossPercentage(bestmodel, X_test, y_test, list(range(0,len(y_test))), accPos, lossPos, percentagePos,packageForFitness_test.freqs, 1) 

    # compute fitness and lower-bound
    realFitness = fitness(packageForFitness_test, minLengthRun)
    if len(indices_of_pos)>0:
        realLBFitness = LB_fitness(packageForFitness_test, minLengthRun, m_AC, indices_of_pos)
        
    # compute predicted LB fitness
    predictions = bestmodel.predict(X_test)
    indices_of_predicted_as_positives = [i for i in range(0, len(X_test)) if predictions[i][1]==1]
    if len(indices_of_predicted_as_positives)>0:
        predictedLBFitness = LB_fitness(packageForFitness_test, minLengthRun, m_AC, indices_of_predicted_as_positives)
        
    print("[TEST]\n[ALL] Loss:", "{:.3f}".format(mean(lossAll)), "\t Acc:", "{:.3f}".format(mean(accAll)))
    print("[POSITIVE ({:.2f}%)] Loss:".format(mean(percentagePos)), "{:.3f}".format(mean(lossPos)), "\t Acc:", "{:.3f}".format(mean(accPos)))
    print("[NEGATIVE ({:.2f}%)] Loss:".format(mean(percentageNeg)), "{:.3f}".format(mean(lossNeg)),"\t Acc:", "{:.3f}".format(mean(accNeg)))
    print("Fitness {:.3f}".format((realFitness)), "\t LB Fitness:", "{:.3f}\n".format((realLBFitness)),"\t Predicted LB Fitness:", "{:.3f}\n".format((predictedLBFitness)))
    
    fake_x, fake_y, fake_packageForFitness, fake_minLengthRun, fake_m_AC  = cleanDataForRF("alignments/mock/"+filename,i,vectorizer_already_trained=True)

    accFake, lossFake = [], []
    accLossPercentage(bestmodel, fake_x, fake_y, range(0,len(fake_y)), accFake, lossFake)
    print("[MOCK] Loss:", "{:.3f}".format(mean(lossFake)), "\t Acc:", "{:.3f}".format(mean(accFake)))
    
    print("-------------------------------------------")



 m_AC= 2
[CROSS-VALIDATION]
[ALL] Loss: 0.011 	 Acc: 0.996
[POSITIVE (0.08%)] Loss: 0.056 	 Acc: 0.982
[NEGATIVE (0.92%)] Loss: 0.007 	 Acc: 0.997
Fitness 0.836 	 LB Fitness: 0.074
 	 LB Fitness: 0.075

Runtime (prediction per trace):0.0000602695
(1441,)
[TEST]
[ALL] Loss: 0.008 	 Acc: 0.998
[POSITIVE (0.08%)] Loss: 0.044 	 Acc: 1.000
[NEGATIVE (0.92%)] Loss: 0.005 	 Acc: 0.998
Fitness 0.837 	 LB Fitness: 0.071
 	 Predicted LB Fitness: 0.073

[MOCK] Loss: 0.206 	 Acc: 0.912
-------------------------------------------

 m_AC= 4
[CROSS-VALIDATION]
[ALL] Loss: 0.020 	 Acc: 0.996
[POSITIVE (0.20%)] Loss: 0.030 	 Acc: 0.994
[NEGATIVE (0.80%)] Loss: 0.017 	 Acc: 0.997
Fitness 0.836 	 LB Fitness: 0.174
 	 LB Fitness: 0.175

Runtime (prediction per trace):0.0000651242
(1441,)
[TEST]
[ALL] Loss: 0.023 	 Acc: 0.999
[POSITIVE (0.20%)] Loss: 0.018 	 Acc: 1.000
[NEGATIVE (0.80%)] Loss: 0.025 	 Acc: 0.998
Fitness 0.837 	 LB Fitness: 0.169
 	 Predicted LB Fitness: 0.170

[MOCK] Loss: 0.391 	 Acc: 0.

# Bonus: Important Features

In [11]:
# ---- (3) prepare the sequences of activities 
data = pd.read_csv("alignments/"+filename,sep = ";", 
                   names = ["traces","tracesWithMoves","runs","runsWithMoves","costs","frequencies"])

traces_to_matrix = data.traces.str.split(":::",expand=True,)
number_of_traces, max_len = traces_to_matrix.shape
print("Number of traces:", number_of_traces, "\nMax len of traces:", max_len) 

# transform the matrix to a serie
traces_to_serie = pd.concat([traces_to_matrix[i] for i in range(0,max_len)], axis=0, 
                                      ignore_index=True, sort=False)
# from the serie, it's easy to get unique words
index_to_word = list(filter(None,(traces_to_serie).unique()))

Number of traces: 4366 
Max len of traces: 176


In [12]:
for feature in zip(index_to_word, np.sort(bestmodel.feature_importances_)):
    print(feature)

('A_SUBMITTED', 0.0)
('A_PARTLYSUBMITTED', 0.0)
('A_DECLINED', 0.00010898079012263087)
('W_Afhandelen leads', 0.00017831727950181467)
('A_PREACCEPTED', 0.00019110727475472446)
('W_Beoordelen fraude', 0.0009586118224845017)
('W_Completeren aanvraag', 0.0031592086765537524)
('A_ACCEPTED', 0.0034073114942701956)
('A_CANCELLED', 0.006056163205674222)
('O_SELECTED', 0.007134492506164547)
('A_FINALIZED', 0.007594912974757332)
('O_CREATED', 0.009294955624499479)
('O_SENT', 0.009479609926060423)
('W_Nabellen offertes', 0.009988496815615235)
('O_SENT_BACK', 0.014086831185175048)
('O_CANCELLED', 0.01463934913061405)
('O_DECLINED', 0.019868295099805436)
('W_Valideren aanvraag', 0.02332422625368348)
('O_ACCEPTED', 0.025264931068749087)
('W_Nabellen incomplete dossiers', 0.027799145415412074)
('A_APPROVED', 0.03182374264532737)
('A_REGISTERED', 0.04446629556848834)
('A_ACTIVATED', 0.25747534402240163)
('W_Wijzigen contractgegevens', 0.4836996712198847)
